In [1]:
import numpy as np
def BB(w0,wt,t,m,z_saved):
    h = 2**m
    T = np.linspace(0,t, h+1)
    W = np.zeros(h+1)
    z = z_saved
    W[0] = w0
    W[h] = wt
    j_max = 1
    
    for k in range(1, m+1):
        i_min = h//2
        i = i_min
        l = 0
        r = h
        for j in range(1, j_max+1):
            a = ((T[r]-T[i])*W[l] + (T[i]-T[l])*W[r]) / (T[r]-T[l])
            b = np.sqrt( (T[i]-T[l])*(T[r]-T[i])/(T[r]-T[l]) )
            W[i] = a + b*z[i]
            i+=h; l+=h; r+=h
        j_max*=2
        h = i_min
    
    return W

In [2]:
import numpy as np
def ELS_antithetic(r,q1,q2,sigma1,sigma2,rho,M,m):
    
    S1_start = 100
    S2_start = 100
    principal = 100    
    
    T = 2
    check_term = 0.5
    check_max = int(T / check_term)
    
    case=6
    S1 = np.zeros(check_max+1)
    S2 = np.zeros(check_max+1)
    payment = np.zeros((3,M)) # To extend the dimension for Y_tilda

    prob = np.zeros((3,case))
    
    for i in range(M):
        
        x1 = np.random.randn(check_max)
        x2 = np.random.randn(check_max)
        z1 = np.zeros(2**m*check_max+1)
        z2 = np.zeros(2**m*check_max+1)
        
        for random_type in [1,-1]:
            S1[0] = S1_start
            S2[0] = S2_start
            
            W1 = np.zeros(check_max+1)
            W2 = np.zeros(check_max+1)

            check_barrier = False
            
            for j in range(1, check_max+1):

                # Random walk
                W1[j] = W1[j-1] + np.sqrt(check_term) * random_type * x1[j-1]
                W2[j] = W2[j-1] + np.sqrt(check_term) * random_type * x2[j-1]

                e1 =  W1[j] - W1[j-1]
                e2 = rho*e1 + (W2[j] - W2[j-1])*np.sqrt(1-rho**2)

                S1[j] = S1[j-1]*np.exp( (r - q1 - (sigma1**2)/2)*check_term + sigma1*e1)
                S2[j] = S2[j-1]*np.exp( (r - q2 - (sigma2**2)/2)*check_term + sigma2*e2)

                # Check Barrier
                if (S1[j] < S1_start * 0.6) or (S2[j] < S2_start * 0.6):
                    check_barrier = True

                # Check early repayment
                if (S1[j] >= S1_start * (0.85 - ((j-1) * 0.05))) and (S2[j] >= S2_start * (0.85 - ((j-1) * 0.05))):
                    payment[random_type+1][i] = principal * (1+0.0625*j) * np.exp(-r * j * check_term)
                    prob[random_type+1][j-1]+=1
                    break

                # Check Maturity repayment
                if j == check_max:
                    if (check_barrier == True) and ( (S1[j] < S1_start * 0.7) or (S2[j] < S2_start * 0.7) ):
                        payment[random_type+1][i] = principal * min(S1[j]/S1_start, S2[j]/S2_start) * np.exp(-r * j * check_term)
                        prob[random_type+1][j+1]+=1
                        break

                    else:
                        S1_BB = np.zeros(2**m+1)
                        S2_BB = np.zeros(2**m+1)
                        S1_BB[0] = S1_start
                        S2_BB[0] = S2_start
                        
                        if random_type == 1:
                            z1 = np.random.randn(2**m*check_max+1)
                            z2 = np.random.randn(2**m*check_max+1)
                        else:
                            z1 = -z1
                            z2 = -z2
                            
                        # Brownian Bridge
                        for k in range(1, check_max+1):
                            
                            W1_BB = BB(W1[k-1], W1[k], 0.5, m, z1[(2**m)*(k-1) : (2**m)*k+1])
                            W2_BB = BB(W2[k-1], W2[k], 0.5, m, z2[(2**m)*(k-1) : (2**m)*k+1])

                            for l in range(1, 2**m+1):
                                e1 = W1_BB[l] - W1_BB[l-1]
                                e2 = rho*e1 + (W2_BB[l] - W2_BB[l-1]) * np.sqrt(1-rho**2)

                                S1_BB[l] = S1_BB[l-1] * np.exp( (r - q1 - sigma1**2/2)*(check_term/2**m) + sigma1 * e1 )
                                S2_BB[l] = S2_BB[l-1] * np.exp( (r - q2 - sigma2**2/2)*(check_term/2**m) + sigma2 * e2 )

                                if(S1_BB[l] < S1_start * 0.6) or (S2_BB[l] < S2_start * 0.6):
                                    check_barrier = True
                                    payment[random_type+1][i] = principal * min(S1[j]/S1_start, S2[j]/S2_start) * np.exp(-r * j * check_term)
                                    prob[random_type+1][j+1]+=1
                                    break

                            if check_barrier:
                                break          
                            else:        
                                S1_BB[0] = S1_BB[2**m]
                                S2_BB[0] = S2_BB[2**m]

                        if check_barrier == False:
                            payment[random_type+1][i] = principal * np.exp(-r * j * check_term)
                            prob[random_type+1][j]+=1
                            break
            
    Y1 = payment[2,:]
    Y2 = payment[0,:]
    value = np.mean((Y1+Y2)/2)
    err = np.std((Y1+Y2)/2) / np.sqrt(M)
    prob = prob[2,:] / M
    
    return value,err,prob,Y1,Y2
from datetime import datetime
r=0.03
q1=0.0
q2=0.0
sigma1=0.3
sigma2=0.4
rho=0.2
M=5000
m=3
t1=datetime.now()
value,err,prob,Y1,Y2=ELS_antithetic(r,q1,q2,sigma1,sigma2,rho,M,m)
t2=datetime.now()
print('value = {:.2f}, err = {:.2f}'.format(value,err))
for i in range(6):
    print('prob[{:d}] = {:{width}.2%}'.format(i+1,prob[i],width=6))
print('Total sum of prob = {:.0%}'.format(sum(prob)))
print('Total computing time = {:f} seconds'.format((t2-t1).total_seconds()))

value = 93.27, err = 0.24
prob[1] = 54.60%
prob[2] = 12.70%
prob[3] =  6.46%
prob[4] =  4.10%
prob[5] =  0.40%
prob[6] = 21.74%
Total sum of prob = 100%
Total computing time = 0.333145 seconds


In [3]:
np.cov(Y1,Y2)

array([[ 721.4366227 , -115.52728792],
       [-115.52728792,  704.20039266]])

In [4]:
np.corrcoef(Y1,Y2)

array([[ 1.        , -0.16208296],
       [-0.16208296,  1.        ]])

In [5]:
np.mean(Y1)

93.02779847301213

In [6]:
np.mean(Y2)

93.50879942664574